#### Let's get started:

STEP 1: MAKE A COPY OF THIS NOTEBOOK AND BEGIN WORKING IN THAT NOTEBOOK

STEP 2: Double-click to get into editing mode. FILL OUT THE FOLLOWING INFORMATION

**_User(s)_**: enter your name(s) here

**_Date_**: enter the date here

**_Description_**: This notebook is analyzing water and surface sedimentd samples from Southwest Colorado and Northern New Mexico in March 2019. This notebook highlights UNIX commands (if you're new), versatility for the Jupyter Notebook and an example dataset to run through.

# Learning the QIIME 2 workflow in the Jupyter Notebook

### Learning objectives

- Understand the steps to run QIIME 2 to analyze data
- Understand the input and output files/directories of QIIME 2
- Generate various plots (taxonomic bar plot, alpha and beta diversity plots) to visualize microbiome results 


### Modules in this workflow

1. Module: Are my sequences of good quality? Getting files uploaded and navigating to the correct directory to run this notebook
2. Module: How do I deal with all of these sequences to be informative? Demultiplexing and quality filtering
3. Module: Who do these sequences belong to? Binning of reads from samples
4. Module: How do my samples compare to one another? How similar or different are they to each other? Summarizing community composition

### How this notebook works
- It is running on the CyVerse Atmosphere's Cloud Computing, the image _[Qiime2 2018_11 - Jupyter Notebook]_ with QIIME 2 and Jupyter Notebook dockerized.
- This notebook reenforces the UNIX Bash Shell commands and introduces the principles of microbiome analysis using QIIME 2.
- This notebook assumes you've run through previous notebooks to set up the system and have the necessary raw data.
- This notebook can be saved, closed and re-started.
- Don't forget to transfer data to the CyVerse Data Store and to suspend your instance to avoid using Atmospheric Units (AUs) and the instance being 'Shelved'.

## Module: Are my sequences of good quality?

In this section of the notebook, we will find the quality of our raw sequence reads in our samples. The raw reads are located in the directory `emp-single-end-sequences`. In order to use QIIME 2, your input data must be stored in QIIME 2 artifacts (i.e. .qza files).

### A. Navigating to the `/scratch` directory - optional if you didn't launch your Jupyter Notebook in root

CyVerse's Atmosphere provides additional storage under `/scratch`. This storage is unique in that it is is faster to store and retrieve data compared to the regular disk storage (normal home directory). Due to its nature, this extra storage will be erased if the instance goes into the shelved state. When an instance is stopped or suspended, the `/scratch` database will preserve the contents (our data). Remember to backup your data to the CyVerse Data Store. 

Visually, `scratch` is accessible under the `root` directory.

**ACTION:** Let's find out our current working directory by running the `pwd` command which stands for `print working directory`.

**ACTION:** Now let's view the contents in our current working directory by running the `ls` command which stands for `list`. Adding the `-F` flag appends indicator. For us, a directory will end with a `/`. 

The data and other files we want to work with are located in the `data` directory. 

**ACTION:** Let's change into the `data` directory by running the `cd` command which stands for `change directory`:

*HINT: After the `cd`, type `d` and press 'tab'. This will autofill possible / available options. Press return to select `data`.*

**ACTION:** Let's check our `current working directory` using the `pwd` command:

> **PRACTICE:** Compare the outputs above. What are the differences?

**ACTION:** Let's practice with the flag `-l` for the `ls` command in the code cell below. We can view information about the files and directories in the `data` directory:

In [ ]:
# these are comments and are not sent instructions to the kernel
# -l option is long read



**ACTION:** Let's practice with the flags `-l` and `-h` combined as `-lh` for the `ls` command in the code cell below. We can view information about the files and directories in the `2018` directory:

In [ ]:
# -h option is human readable
# you can combine these options together -lh



For the `ls -l` and `ls -lh`, we are able to list the contents of the directory in the long read and long read human readable. This gives seven fields with various information. Each field is described below:

*1st column* gives information about permissions. Those permissions means read (`r`), write (`w`) and execute (`x`), and comes in three different fields that indicate the permission of: 
```
  1st 2nd 3rd 4th
    - --- --- ---
    d rwx rwx rwx
    
    first: d is directory
    OWNER: second
    GROUP: third
    EVERYONE: fourth
```

*2nd column* specifies the number of links or directories inside this directory.  

*3rd column* The user that owns the file, or directory.  

*4th column* The group that file belongs to, and any user in that group will have the permissions given in the third field over that file.  

*5th column* The size in bytes. Using the `-lh` flags together will have the output in `K,M,G` for a easier viewing of the byte size that is more human readable.

*6th column* The date of last modification.  

*7th column* The name of the file with its filetype information

> **PRACTICE:** How could this be helpful with many files?


**ACTION:** Let's look at the contents within the `emp-single-end-sequences` directory using `ls -lh` in the code cell below:

In [ ]:
ls -lh emp-single-end-sequences

> **PRACTICE:** Discuss the following questions about the files in `emp-single-end-sequences` directory.
>
> 1. What do you notice about the filenames?
> 2. What are the file formats?
> 3. What are the sizes of these files?
> 4. How many files are listed?

-------------------
> The files in `emp-single-end-sequences` directory are the raw sequence files. FASTQ files are saved compressed in the GNU zip format (an open source file compression program), indicated by the `.gz` file extension. The sample info is the barcode used for those samples.
>
> Their file sizes are `M` which stands for `megabyte` and `K` which stands for `kilobyte`. The file `NCR1` is `269K` which is `0.269 M`. 

### B. What do our `.fastq.gz` files look like?

The FASTQ files hold important information about the sequence and quality of the sequences. The FASTQ files to be analyzed are gzipped or referred to as compressed. 

`.fastq.gz` files are used because they can be transferred faster. The QIIME 2 plugin `demux` can handle the this file type: gzipped `.fastq.gz` for downstream analysis.

Illumina sequencing instruments generate per-cycle base call (BCL) files at the end of the sequencing run. The BCL files are translated into FASTQ files generated by the proprietary program from Illumina. The resulting samples are in the CASAVA demultiplexed 1.8 FASTQ Format.

The files were are working with are in the multiplexed format means there is one `fastq.gz file` that contains all the reads for each sample in the study.


> **PRACTICE:**
> Why do you think we use a compressed format? 

--------------
>
> A typical FASTQ file contain hundreds to millions of sequences and takes up dozens of bytes on a disk. When these files are compressed with `gzip`, their sizes are reduced in more than 10 times (`zip` format is less efficient). Keep in mind the total number of samples and how that increased the need for disk space.

We can "peek" into a `.fastq.gz` by using two different commands called `zcat` (linux) and `head`. These are useful as we can see the contents without **unzipping** the file! Note: if you're running on a MacOS, `gzcat` will be used.

The file we will peek into is `HHMI_PC_Rep1_TATAGCGA-ACTATCTG_L001_R2_filtered.fastq.gz`.

```
Usage:

zcat FILENAME.fastq.gz | head -4
```

Here we use the pipe `|` to pass the output from `zcat` to `head` to view the top 4 lines of the read.

**ACTION:** Use `zcat` and `head` to view the first four lines of the `HHMI_PC_Rep1_TATAGCGA-ACTATCTG_L001_R2_filtered.fastq.gz` file in the cell below.

In [ ]:
zcat

> _The line `gzip: stdout: Broken pipe` at the bottom of the output may appear. This is an issue within the Jupyter Notebook. Don't worry about this issue!_

-----


A `.fastq` file stores both a nucleotide sequence and its corresponding quality scores. A `.fastq` file normally uses four lines per sequence:

**Line 1** begins with a `@` character and is followed by a sequence identifier (specific to the sequencing machine)

**Line 2** is the raw sequence

**Line 3** begins with a '+' character and is optionally followed by the same sequence identifier (and any description) again.


**Line 4** encodes the quality values for the sequence in Line 2, and must contain the same number of symbols as letters.

> **PRACTICE:** Do you find each of these lines in the preview?

For more information about FASTQ file format, check out [FASTQ format](http://maq.sourceforge.net/fastq.shtml).
 

### C. Check the QIIME program

**_NOTE: So far we haven't touched QIIME 2. We have only uploaded our data on the CyVerse Atmosphere Virtual Machine and are setting up our file system. To run the remainder of the notebook, we will need to be in the `/scratch` directory to begin working with our data._**

### What is QIIME 2?

[QIIME 2](https://qiime2.org/) is a microbiome analysis package with a focus on data and analysis transparency. QIIME 2 enables researchers to start an analysis with raw DNA sequence data and finish with publication-quality figures and statistical results.


![](https://docs.qiime2.org/2019.1/_images/overview.png)


Running QIIME 2 on CyVerse's Atmosphere is set-up in a Docker container. We run the `alias` command below to initialize QIIME 2.

alias qiime='docker run -t -i -u $(id -u):$(id -g) -v $(pwd):/data  -v /scratch/tmp:/tmp qiime2/core:2018.11 qiime'

Let's start by running one QIIME 2 command `info`. 

```
Usage:

qiime [command]
```
```info``` will give the system version and available plugins available. This is useful information to understand what types of analyses can be run. 

**ACTION:** Run the command in the code cell below:

In [ ]:
qiime info

> If this error pops up:
>
>```
Command 'qiime' not found, but can be installed with:
apt install qiime
Please ask your administrator
```
> QIIME 2 is not set-up yet. Re-run the `alias` command.

> *What is a plugin?*
>
>QIIME 2 microbiome analyses are made available to users via plugins. A plugin is a Python 3 package developed by people to run microbiome analysis in the [QIIME 2 program/framework](https://docs.qiime2.org/2018.11/concepts/#plugins).
>
>To perform analyses with QIIME 2, one or more plugins provide the specific analyses you are interested in. For example, if you want to demultiplex your raw sequence data, you might use the demux plugin, or if you’re wanting to perform alpha- or beta-diversity analyses, you could use the diversity plugin.
>
```
dada2: 2018.11.0
demux: 2018.11.0
diversity: 2018.11.0
emperor: 2018.11.0
feature-classifier: 2018.11.0
feature-table: 2018.11.0
metadata: 2018.11.0
phylogeny: 2018.11.0
sample-classifier: 2018.11.0
taxa: 2018.11.0
```

There are many more [QIIME 2 plugins available](https://docs.qiime2.org/2019.1/) and community-driven. 

Let's begin working with the program QIIME 2!

> **Obtaining and importing data**
>
> All data that is used as input to QIIME 2 is in form of QIIME 2 artifacts, which contain information about the type of data and the source of the data. 
>
> Data produced by QIIME 2 exist as [QIIME 2 artifacts](https://docs.qiime2.org/2018.11/concepts/#data-files-qiime-2-artifacts). A QIIME 2 artifact contains data and metadata. The metadata describes things about the data, such as its type, format, and how it was generated (provenance). A QIIME 2 artifact typically has the `.qza` file extension when stored in a file.
>
> Every artifact generated by QIIME 2 has a [semantic type](https://docs.qiime2.org/2018.11/concepts/#semantic-types) associated with it. Defining semantic types allows us to ensure that the data that is passed to an action is meaningful for the operation that will be performed. We give a semantic to run a plug-in. 

### D. Let's import these sequence data files into a QIIME 2 artifact.

Our data has not been demultiplexed (barcodes and adapters removed). 

The semantic type of this QIIME 2 artifact is EMPPairedEndSequences. EMPPairedEndSequences QIIME 2 artifacts contain sequences that are multiplexed, meaning that the sequences have not yet been assigned to samples (hence the inclusion of `forward.fastq.gz`, `reverse.fastq.gz` and `barcodes.fastq.gz` files, where the `barcodes.fastq.gz` contains the barcode read associated with each sequence in `forward.fastq.gz.` and `reverse.fastq.gz`). The importing of other types of data can be found on the [Importing Data](https://docs.qiime2.org/2018.11/tutorials/importing/) page.

To import our data, we use the QIIME command `tools import`. To check the usage:

In [ ]:
qiime tools import --help

The plugin requires the following arguments `[ARGS]` : `input-path`, `type` and `output-path`.

**ACTION:** Let's import the `.fastq.gz` files from the directory `input-path` `emp-paired-end-sequences` and place them into the `emp-paired-end-sequences.qza` artifact.

> Note: In these code cells, you'll notice at the end of each line is a backslash `\`. This is an indicator to continue on to the next line. The last line does not contain a backslash. In terminal, the backslash is unnecessary.

In [ ]:
qiime tools import \
--type EMPPairedEndSequences \
--input-path emp-paired-end-sequences \
--output-path emp-paired-end-sequences.qza

### E. Incorporating your metadata

Metadata provides the key to gaining biological insight from your data. In QIIME 2, [sample metadata](https://docs.qiime2.org/2018.11/tutorials/metadata/) may include technical details, such as the DNA barcodes that were used for each sample in a multiplexed sequencing run, or descriptions of the samples, such as which subject, time point, and site each sample came from. 

What is the metadata file? This is a tabular `.tsv` tab separated value file.

Go to the home directory to view `metadata-flc-2019.tsv`.

Metadata is information about your samples (e.g. date collected, depth, temperature, pH, etc). This information should be contained within a single file that has samples as rows and variables as columns.

Here is an example preview of a metadata file for :

|#SampleID | BarcodeSequence | LinkerPrimerSequence | depthCM | Replicate | CollectionDate |pH| TempC |

|-------- |:-------------:|:-------------:| -----:|-----:|:-------------:| -----:|
|River32ndRep1 |CCTCGCATGACC	|GTGCCAGCMGCCGCGGTAA| 10	|	1	| 20180329|7.7|7|
|River32ndRep2 |GGCGTAACGGCA	|GTGCCAGCMGCCGCGGTAA| 10	|	2	| 20180329|7.7|7|
|River32ndRep3 |GCGAGGAAGTCC	|GTGCCAGCMGCCGCGGTAA| 10	|	3	| 20180329|7.2|7|
|Sediment32ndRep1  |CAAATTCGGGAT	|GTGCCAGCMGCCGCGGTAA| 5	|	1	| 20180329|7.2|4|
|Sediment32ndRep2 |TTGTGTCTCCCT	|GTGCCAGCMGCCGCGGTAA| 5	|	2	| 20180329|7.8|4|
|Sediment32ndRep3 |CAATGTAGACAC	|GTGCCAGCMGCCGCGGTAA| 5	|	3	| 20180329|7.8|4|
|River9thRep1 |AACCACTAACCG	|GTGCCAGCMGCCGCGGTAA| 10	|	1	| 20180329|8.0|9|
|River9thRep2 |CCTCGCATGACC	|GTGCCAGCMGCCGCGGTAA| 10	|	2	| 20180329|8.1|9|
|River9thRep3 |CCTCGCATGACC	|GTGCCAGCMGCCGCGGTAA| 10	|	3	| 20180329|8.2|9|


In order to use QIIME2, you must have your spreadsheet correctly formatted. 

Setting up this file up as a google sheet is one way. To check whether your file is correctly formatted, use the [Keemei plugin](https://keemei.qiime2.org/) for google sheets. Once Keemei says your spreadsheet is correctly formatted for QIIME 2, you’re ready to proceed!

**ACTION:** Using the `qiime tools inspect-metadata` plugin with the input `.tsv` file to inspect and print the metadata file.

In [ ]:
qiime tools inspect-metadata filename

**ACTION:** Using the `qiime metadata tabulate` plugin with the input `.tsv` file to generate a visualization file.

In [ ]:
qiime metadata tabulate \
  --m-input-file
  --o-visualization tabulated-metadata-flc-2019.qzv

### C. Viewing our .qzv files

We are not working in an environment that allows us to see interactively our visualization files in this notebook. We need to download the `.qzv` files to our local computer to view them.

1. Go to the Home tab.
2. To download the `.qzv` files to your computer, find the `.qzv` file and right-click. The `Download` option will become available.
3. Launch the website: https://view.qiime2.org/.
4. Drag and Drop `.qzv` file to view plot.

> Make a new directory / folder on your local computer desktop to store all the `.qza` and `.qzv` files that are generated through this tutorial.  --o-visualization tabulated-sample-metadata.qzv

### D. Viewing the `tabulated-metadata-flc-2019.qzv` file

> QUESTION:
> 
> Based on the table in tabulated-sample-metadata.qzv, how many samples are associated with `gardenbed`? How many samples are associated with a `depth of 5 cm`? 
>
>_Hint: use the search box and/or the column sorting options to assist with this query._
>
> 
> PRACTICE:
>
> Take note of the columns: Treatment, pH, and temperatureC. We will be using these downstream for Diversity Analysis.

### F. Let's demultiplex our sequences 

To demultiplex sequences we need to know which barcode sequence is associated with each sample. This information is contained in the sample metadata file. 

```
Usage: qiime demux emp-paired [OPTIONS]
Options:
  --i-seqs ARTIFACT PATH EMPPairedEndSequences
  --m-barcodes-file MULTIPLE FILE
  --m-barcodes-column MetadataColumn
  --o-per-sample-sequences ARTIFACT PATH
```
The demux.qza QIIME 2 artifact will contain the demultiplexed sequences.

In [ ]:
qiime demux emp-paired \
--m-barcodes-file metadata-flc-2019.tsv \
--m-barcodes-column BarcodeSequence \
--i-seqs emp-paired-end-sequences.qza \
--o-per-sample-sequences demux.qza

#### Output artifacts
```
demux.qza
```

If there is an error, consult the instructor. A space, incorrect file path could be the possible issue. If the command ran successfully, a green statement will be printed. The output `.qza` is a QIIME 2 artifact contains data and metadata.

### G. Sequence quality control

After demultiplexing in QIIME 2, it’s useful to generate a summary of the demultiplexing results. This allows you to:

- determine how many sequences were obtained per sample
- get a summary of the distribution of sequence qualities at each position in your sequence data

To summarize the counts per sample for all samples and generate interactive positional quality plots, we will use the QIIME 2 plugin `demux`.

```
Usage: qiime demux summarize [OPTIONS]

qiime demux summarize \
--i-data demux.qza \
--o-visualization demux.qzv
```

The two options required are `--i-data` and `--o-visualization`. The option `--i-data` is the input file we previously generated `demux.qza`. The option `--o-visualization` will be the resulting output filename, a `.qzv` file which is a QIIME 2 visualization file

[Visualization files](https://docs.qiime2.org/2018.11/concepts/#data-files-visualizations) are another type of data generated by QIIME 2. Visualizations contain similar types of metadata as QIIME 2 artifacts. Visualizations are standalone information that can be archived or shared with collaborators through [qiime2.org](qiime2.org).

**ACTION:** Enter the command below to generate the `demux.qzv` file:

In [ ]:
qiime demux summarize \
--i-data demux.qza \
--o-visualization demux.qzv

#### Output Visualizations
```
demux.qzv
```
If there is an error, consult the instructor. If the command ran successfully, a green statement will be printed. The output `.qzv` is a QIIME 2 artifact contains data and metadata.

### H. Viewing our `.qzv` files

We are not working in an environment that allows us to see interactively our visualization files in this notebook. We need to download the `.qzv` files to our local computer to view them.

1. Go to the Home tab.
2. To download the `.qzv` files to your computer, find the `.qzv` file and right-click. The `Download` option will become available.
3. Launch the website: https://view.qiime2.org/.
4. Drag and Drop `.qzv` file to view plot.

> Make a new directory / folder on your local computer desktop to store all the `.qza` and `.qzv` files that are generated through this tutorial.

### I. Interpreting results of `.demux.qzv`

The QIIME 2 View webpage will open to the plot. In the header bar, there are the tabs: *Visualization*, *Peek*, and *Provenance*. We will focus on the *Visualization* tab for now.

Below the header bar are two tabs: `Overview` and `Interactive Quality Plot`. 

** In the Overview Tab** 

- Table : Demultiplexed sequence counts summary : *This is the summary of all samples*

> **PRACTICE:**
>
> Review the table and answer the following questions: 
>
> A. What is the the lowest number of sequences found in a sample? Write the number of sequences.
>
> B. What is the highest number of sequences found in a sample? Write the number of sequences.
>
> C. Write down the average number of sequences found?
>
> D. What is the total number of sequences found in all the samples?
>
>
> - Plot : *The Number of sequences (`x-axis`) and Frequency (`y-axis`)*
>
>
> This is a bar plot showing the **distribution of sequences** found in each sample.
>
> E. Discuss the distribution plot with a partner. Think about what type of sample may contain more sequences. Write up a short summary describing the number of sequences in each samples. For example, 8 samples contain ~100000 sequences and 3 samples with less than ~25000 sequences.
>
>
> - Table : Per-sample sequence counts : *The number of sequences found in the sample*
>
> F. What sample name has the highest amount of sequences?
>
> G. What sample name has the lowest amount of sequences?
>
> H. In a `negative control` (`NC`), should we expect sequences? Why or why not?
> 
> I. In a `positive control`, (`PC`), should we expect sequences? Why or why not?
> 

**In the Interactive Quality Plot Tab**

*What is a good quality score?*

A quality score (Q-score) is a prediction of the probability of an error in base calling. Q scores are one of the most common metrics for assessing sequencing data quality. During a sequencing run, a quality score is assigned to each base call for every cluster, on every tile, for every sequencing cycle. A high quality score implies that a base call is more reliable and less likely to be incorrect. Low Q scores can lead to increased false-positive variant calls, resulting in inaccurate conclusions and higher costs for validation experiments. In the table below, the higher quality score results in a probability the base call is incorrect.

|Quality Score | Probability of Incorrect Base Call | Base Call Accuracy
| ------------- |:-------------:| -----:|
| 10  | 1 in 10 | 90%|
| 20 | 1 in 100 | 99%|
| 30 | 1 in 1,000 | 99.9%|
| 40 | 1 in 10,000 | 99.99%|
| 50 | 1 in 100,000 | 99.999%|

Q score of 20 (Q20) to a base, will have an incorrect base call probability of 1 in 100, meaning that
every 100 bp sequencing read will likely contain an error. 

> **PRACTICE**
>
> The plot on the `x-axis` is the order of the base. Our sequenced reads are 250 base pairs in length. The `y-axis` is the quality score associated to the base pair. Due to limitations of sequencing technologies, the sequenced read quality score tends to drop. These plots are commonly generated to view sequenced reads quality score.
>
> Using the cursor, drag over a section of the plot. Zooming in, we see [box and whisker plot](https://www.khanacademy.org/math/statistics-probability/summarizing-quantitative-data/box-whisker-plots/a/box-plot-review). These plots depicting the spread of all the data points. The vertical lines extending from the boxes (whiskers) indicating variability outside the upper and lowest data point. The line in the square box is the median. The square box is the quality score range at the base pair location. There are reads that have very low scores we may not want to use.
>
> J. If the range is the difference between the upper and lowest data point (whisker), calculate the range quality score at the sequence length of 228. 
>
> K. Based on the plots you see in `demux.qzv`, what values would you choose trim your reads? (When do you see a majority of your samples to have a lower quality score?)
>
> Note: The values for trimming will be used in the next section for `[ARGS]`: `--p-trunc-len` and `--p-trim-left`.
>
> L. Based on the plots you see in `demux.qzv`, what values would you choose for `--p-trunc-len` and `--p-trim-left` in this case?

> ANSWERS FROM ABOVE: 
>
> A. Lowest sequence counts (from table the minimum) found in a sample - 2101
>
> B. Highest sequence counts (from table the maximum) found in a sample - 117550
>
> C. Average sequence counts (from table the mean) - 53725
>
> D. Total sequences of all samples - 3008636
>
> E. There are 2 samples that contain over 300000 reads. The 20 samples contain less than 50000 reads in their sample. 6 samples contain over 100000 reads but less than 300000.
>
> F. What sample name has the highest amount of sequences? 32WR3 with 117550
>
> G. What sample name has the lowest amount of sequences? NTC2 with 2101
>
> H. In a `negative control` (`NC`), should we expect sequences? Why or why not? In a perfect world, we wouldn't see any reads because there shouldn't be any microbial DNA amplified and sequenced from our samples. The reagent kits do contain microbial DNA, but only a small amount. The molecular grade water used as our negative control for sample preparation should not contain any microbial DNA either but contamination can occur from the air or even pipetting. Thus we should expect sequences but it should be on the lower end of sequence counts.
>
> I. In a `positive control`, (`PC`), should we expect sequences? Why or why not? The positive control samples contain microbial DNA. We hope to extract the DNA, amplify the 16S rRNA v4 region and sequence reads.
>
> J. Range = 39 - 16 = 23. So our quality score fall in the range of 16 and 39 at sequence read length of 150.
>
> K. The V4 region length is approximately 254 bp. The interactive plot has reads with median quality score of 37 around 150 bp. With paired end reads, if you trim too short, the downstream analysis may lose information for clustering and taxonomic analysis. The opposite is true if lower quality scoring reads are included.
>
> L. As a starting point, chooseing `--p-trunc-len 150` and `--p-trim-left 13`.

# MAGGIE START HERE

## Module: How do I deal with all of these sequences to be informative?

### A. Quality filtering sequence data, denoising and clustering

In our `fastq.gz` files, there are many sequences that we cannot process one-by-one. To make sense of this thousands of lines, we can identify similar sequences and get rid of _bad_ sequences.

To clean up and cluster our reads, we will use the `DADA2` workflow. [DADA2](https://www.ncbi.nlm.nih.gov/pubmed/27214047) is a pipeline for detecting and correcting (where possible) Illumina amplicon sequence data. The first step is to denoise, remove or correct noisy reads. This type of quality control process will additionally filter any phiX reads (commonly present in marker gene Illumina sequence data) that are identified in the sequencing data, and will filter chimeric sequences. 

All sequences to filter and cluster sequences:
```
Plugin: dada2 denoise-paired
Input:  demux.qza
Output: table.qza (artifact)
        rep-seqs.qza (artifact)
```
The `dada2 denoise-paired` method requires one parameters used in quality filtering: `--p-trim-left` and `--p-trunc-len` which truncates each sequence at position `n`. This allows the user to remove low quality regions of the sequences. To determine what values to pass for this parameters, review the Interactive Quality Plot tab in the `demux.qzv` file that was generated by qiime demux summarize above.

For our dataset, `dada2` runs for a long time. We will need to submit the job in a different format than in a code cell. If we close the notebook, all jobs end. To run an uninterrupted `dada2` job, we move into the `Terminal` to submit the job and it will keep running in the background.

Follow these steps:

1. Go to the home menu (in the other tab).
2. On the home menu, go to the right side of the screen towards the `New` button and select `Terminal`.
3. A new window will pop-up. After the prompt sign `$` type `screen`.
4. Text will appear. Press the `enter` key twice.
5. Copy the alias line (below) and paste after the `$` prompt:
```
alias qiime='docker run -t -i -u $(id -u):$(id -g) -v $(pwd):/data  -v /scratch/tmp:/tmp qiime2/core:2018.11 qiime'
```
6. Run the alias line.
7. After the command prompt `$` type `pwd`.
8. We need to be in the directory `/scratch/data`. If you are in root `/`. you can type `cd scratch/data`.
9. Type `pwd` to see if you are in the `/scratch/data` directory.
10. If you are in the `/scratch/data` directory, type the command below or copy to the command prompt `$`:

In [ ]:
qiime dada2 denoise-paired \
  --i-demultiplexed-seqs demux.qza \
  --p-trim-left-f 13 \
  --p-trim-left-r 13 \
  --p-trunc-len-f 150 \
  --p-trunc-len-r 150 \
  --o-table table-dada2.qza \
  --o-representative-sequences rep-seqs-dada2.qza \
  --o-denoising-stats denoising-stats-dada2.qza

#### Output Artifacts
```
--o-representative-sequences rep-seqs-dada2.qza \
--o-table table-dada2.qza \
--o-denoising-stats stats-dada2.qza
```

**ACTION:** Check if the output artifacts were generated into current working directory using the command `ls -lt`:

In [ ]:
ls -lt

In [ ]:
qiime tools inspect-metadata mappingfile_soil2018.tsv

**ACTION:** Using the `qiime metadata tabulate` plugin with the input `.tsv` file to generate a visualization file.

In [ ]:
qiime metadata tabulate \
  --m-input-file mappingfile_soil2018.tsv \
  --o-visualization tabulated-sample-metadata.qzv

### B. FeatureTable and FeatureData summaries

After the quality filtering step and clustering completes, you’ll want to explore the resulting data! The two artifacts (`table-dada2.qza` and `rep-seqs-dada2.qza`) generated are **important** and can be used in other programs and downstream analyses in QIIME 2. One way to make sense of all the data is to create visual summaries.

The feature table is a matrix of samples and observations. For example, the number of times a "feature" is observed in each sample in a data set. A feature can be aperational taxonomic units (OTUs) or amplicon sequence variants (ASVs).

The `feature-table summarize` command will give you information on how many sequences are associated with each sample and with each feature, histograms of those distributions, and some related summary statistics.

**ACTION:** Generate a visualization file summarizing the table from the DADA2 analysis:

In [ ]:
qiime feature-table summarize \
  --i-table table-dada2.qza \
  --o-visualization table-dada2.qzv \
  --m-sample-metadata-file metadata-flc-2019.tsv

The `feature-table tabulate-seqs` command will provide a mapping of feature IDs to sequences, and provide links to easily BLAST each sequence against the NCBI nt database. The latter visualization will be very useful later in the tutorial, when you want to learn more about specific features that are important in the data set.

**ACTION:** Generate a visualization file of tabulated sequences:

In [ ]:
qiime feature-table tabulate-seqs \
  --i-data rep-seqs-dada2.qza \
  --o-visualization rep-seqs-dada2.qzv

#### Output Visualizations
```
table-dada2.qzv
rep-seqs-dada2.qzv
```

### F. Viewing our `table.qzv` and `rep-seqs.qzv` files

We are not working in an environment that allows us to see interactively our visualization files in this notebook. We need to download the `.qzv` files to our local computer to view them.

1. Go to the Home tab.
2. To download the `.qzv` files to your computer, find the `.qzv` file and right-click. The `Download` option will become available.
3. Launch the website: https://view.qiime2.org/.
4. Drag and drop `.qzv` file to view plot.

> Make a new directory / folder on your local computer desktop to store all the `.qza` and `.qzv` files that are generated through this tutorial.  --o-visualization tabulated-sample-metadata.qzv

Now let's **PRACTICE:**

> Load the `table-dada2.qzv` file. 
>
> A. In the Overview Tab: Table Summary, what is the total number of features are found? How many samples are in this dataset?
>
> B. In the Overview Tab: The histogram of the Frequency per sample, the `x-axis` is the frequency per sample with the `y-axis` is the number of samples. How many samples have a frequency over 150,000? 
>
> C. From the Interactive Sample Detail: What `sampling-depth` value will you choose for the input for `--p-sampling-depth` parameter? This value will be used when running `qiime diversity core-metrics-phylogenetic` later. Think about if we want to include ALL the samples.
> 
> _HINT: use the pull-down menu to view the plot based on the sample metadata. Take the bar and slide it along to identify the number of sequences to use from the samples._
>
> D. From the value chosen from `--p-sampling-depth`, how many samples will be excluded from your analysis based on this choice?
>
> _HINT: scroll down as they may be highlighted in red._
>
> E. From the Feature Detail Tab: Take a look at how the table is formed. The first column in a generated feature-id name (coded string of letters/numbers) that represent one sequence. The second column is the total frequency it is found. The third column is how many samples it is found in. This is an extensive list to go through and sort. Thankfully many tools have been written to filter and sort this table. Copy the fifth FeatureID. (i.e. `30a8fac2ccac130502241563e19d75eb`)
>
> Note: For each DADA2 run, the feature IDs will be different. 
>
> Load the `rep-seqs-dada2.qzv` file.
>
> F. Here, you can view the actual sequence of the Feature by searching the FeatureID. In the Internet Browser, use the Find tool (CRTL+F or CMD+F) to search the FeatureID. Click on that sequence (a link) to open up BLAST. Run a quick BLAST and view the report. What is the matching nucleotide sequence identified?
>
> Imagine doing this for each FeatureID. How long would it take to go through all the features like this? Thankfully the next section does that for us!

### G. Wrap up
Don't forget to transfer data to the CyVerse Data Store and to suspend your instance to avoid using Atmospheric Units (AUs) and the instance being 'Shelved'.

> Load the `table-dada2.qzv` file. 
>
> A. In the Overview Tab: Table Summary, what is the total number of features are found? How many samples are in this dataset? 17,869 features and 56 samples
>
> B. In the Overview Tab: The histogram of the Frequency per sample, the `x-axis` is the frequency per sample with the `y-axis` is the number of samples. How many samples have a frequency over 60,000? 21
>
> C. From the Interactive Sample Detail: What `sampling-depth` value will you choose for the input for `--p-sampling-depth` parameter? This value will be used when running `qiime diversity core-metrics-phylogenetic` later. Think about if we want to include ALL the samples. 516
> 
> _HINT: use the pull-down menu to view the plot based on the sample metadata. Take the bar and slide it along to identify the number of sequences to use from the samples._
>
> D. From the value chosen from `--p-sampling-depth`, how many samples will be excluded from your analysis based on this choice? 21,503
>
> _HINT: scroll down as they may be highlighted in red._
>

## Module: Who do these sequences belong to?

In the previous section, we sorted out and clustered the sequences. These sequences do not have any information associated with them. In this section, we will first construct a phylogenetic tree and then second assign taxonomy to the sequences using the phylogenetic tree information.

Reconstructing the phylogeny of a group of individual microbes is useful for many reasons. The most obvious of these is understanding the evolutionary relationship between a group of organisms. 

<img src="https://raw.githubusercontent.com/gregcaporaso/An-Introduction-To-Applied-Bioinformatics/master/book/fundamentals/images/Pace_Big_Tree.png" width="500">

**Figure**: A hypothesis of evolutionary relationships between the three domains of life: Bacteria, Eucarya, Archaea. This image was created by the Norman Pace Laboratory.

Phylogenetic trees use a variety of algorithms. These algorithms compare a set of features of organisms and infer the evolutionary distance between those organisms based on the similarity of their features.

### A. Tree-thinking

Phylogenetic trees show the relationships.

**Figure**: (from [Meiser et al, 2010](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3092386/)) These phylogenetic trees show the relationships of four species (A, B, C, and D). Each tree represents the same relationships of the four species: species C and D are the closest relatives, species B is equally related to both C and D, and species A is the outgroup.

<img src="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3092386/bin/nihms233238f1.jpg" width="500">

A) A large arrow indicates the direction of time from the past to the present. The most recent common ancestor (MRCA) of all four species is indicated by a circle and small arrow. An arrow indicates the root of the tree. The tree on the right is a diagonal representation of the evolutionary relationships.

B) Two clades are indicated on each tree: clade 1 consists of species C and D, and it is nested within clade 2, consisting of species B, C, and D.

C) A fifth species (Z) has been added to the phylogeny, but the relationships of species A, B, C, and D have not changed.

D) A circles-within-circles diagram can represent the nested hierarchical relationships of species A, B, C, and D.

E) The branches of the tree have been rotated around various nodes, but they still depict the same evolutionary relationships

### B. Generate a tree for phylogenetic diversity analyses

From our sequences, we are constructing a large phylogenetic tree. Then will use it in downstream analyses.

QIIME 2 supports several phylogenetic diversity metrics, including Faith’s Phylogenetic Diversity and weighted and unweighted UniFrac. In addition to counts of features per sample (i.e., the data in the `FeatureTable[Frequency]` QIIME 2 artifact), these metrics require a rooted phylogenetic tree relating the features to one another. This information will be stored in a `Phylogeny[Rooted]` QIIME 2 artifact. The following steps will generate a `rooted-tree.qza`.

The next four steps, you will go through various steps to generate a phylogenetic tree:
1. generate a multiple sequence alignment using Mafft
2. mask (remove highly variable positions) that may add noise to the multiple sequence alignment
3. construct a phylogenetic tree based on nucleotide sequence
4. root the phylogenetic tree

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs-dada2.qza \
  --o-alignment aligned-rep-seqs-dada2.qza \
  --o-masked-alignment masked-aligned-rep-seqs-dada2.qza \
  --o-tree unrooted-tree-dada2.qza \
  --o-rooted-tree rooted-tree-dada2.qza

> _WHATS HAPPENING?_
>
> The above command will perform the following steps.
>
> First, we perform a multiple sequence alignment (MSA) of the sequences in our `FeatureData[Sequence]` to create a `FeatureData[AlignedSequence]` QIIME 2 artifact. 
>
> Here we do this with the `mafft` program. [MAFFT](https://mafft.cbrc.jp/alignment/software/) stands for Multiple Alignment using Fast Fourier Transform.  
>
> Generate a sequence alignment
>
> ```
> Plugin: qiime alignment mafft
> Input:  rep-seqs.qza (artifact)
> Output: aligned-rep-seqs.qza (artifact)
> ```
>Next, we mask (or filter) the alignment to remove positions that are highly variable. These positions are generally considered to add noise to a resulting phylogenetic tree.
>
>Next, we’ll apply the tool [FastTree](http://www.microbesonline.org/fasttree/) to generate a phylogenetic tree from the masked alignment. FastTree infers approximately-maximum-likelihood phylogenetic trees from alignments of nucleotide sequences. 
>
>The FastTree program creates an unrooted tree, so in the final step in this section we apply midpoint rooting to place the root of the tree at the midpoint of the longest tip-to-tip distance in the unrooted tree.

#### Output artifacts
```
aligned-rep-seqs-dada2.qza 
masked-aligned-rep-seqs-dada2.qza
unrooted-tree-dada2.qza 
rooted-tree-dada2.qza
```

We will not be generating a visualization file for the phylogenetic tree. However, we will be using the output file in downstream analysis. We will come back to this in the next module.

### C. Taxonomic analysis

Next steps: 
- we will take denoised sequences `rep-seqs.qza` and assign taxonomy (phylum -> class -> genus) to each sequence
- we will explore the taxonomic composition of the samples
- understand how to relate that to sample metadata. *This is where data about sample collection becomes important!*
- Compare the two available taxonomic databases: GreenGenes and SILVA.

The first step in this process is to assign taxonomy to the sequences in our `FeatureData[Sequence]` QIIME 2 artifact. We’ll do that using a pre-trained Naive Bayes classifier and the `q2-feature-classifier` plugin. The first classifier was trained on the `Greengenes 13_8 99% OTUs`.

> *What is the annotated databases?*
>
> where the sequences have been trimmed to only include 250 bases from the region of the 16S that was sequenced in this analysis (the V4 region, bound by the 515F/806R primer pair). 

We’ll apply this classifier to our sequences, and we can generate a visualization of the resulting mapping from sequence to taxonomy.

**ACTION:** First, let's download the trained Greengenes classifier from [QIIME 2.org](https://docs.qiime2.org/2019.1/data-resources/#taxonomy-classifiers-for-use-with-q2-feature-classifier):

In [ ]:
wget \
  -O "gg-13-8-99-515-806-nb-classifier.qza" \
  "https://data.qiime2.org/2018.11/common/gg-13-8-99-515-806-nb-classifier.qza"

In [ ]:
qiime feature-classifier classify-sklearn \
  --i-classifier gg-13-8-99-515-806-nb-classifier.qza \
  --i-reads rep-seqs-dada2.qza \
  --o-classification taxonomy-dada2.qza

Output artifacts:

`taxonomy-dada2.qza`

`gg-13-8-99-515-806-nb-classifier.qza`

Next, we use `metadata tabulate` plugin to generate a `.qzv` file. We searched the FeatureID (i.e. 30a8fac2ccac130502241563e19d75eb) and had only the sequence. Now if we search for the FeatureID, we will see a taxonomic assignment.

In [ ]:
qiime metadata tabulate \
  --m-input-file taxonomy-dada2.qza \
  --o-visualization taxonomy-dada2.qzv

Output visualizations:

`taxonomy-dada2.qzv`

> Question
>
> Recall that our `rep-seqs.qzv` visualization allows you to easily BLAST the sequence associated with each feature against the NCBI nt database. Using that visualization and the taxonomy.qzv visualization created here, compare the taxonomic assignments with the taxonomy of the best BLAST hit for a few features. How similar are the assignments? If they’re dissimilar, at what taxonomic level do they begin to differ (e.g., species, genus, family, ...)?


Next, we can view the taxonomic composition of our samples with interactive bar plots. Generate those plots with the following command and then open the visualization.

Scaling up the visualization, the various taxonomic assignments can be viewed as a taxonomic composition of our samples. In QIIME 2 View, we will use interactive bar plots. We need to generate the `.qzv` taxa bar plots.

```
Plugin: qiime taxa barplot
Input:  table.qza (artifact)
        taxonomy.qza (artifact)
        mappingfile_soil2018.tsv
Output: taxa-bar-plots-hhmi.qzv (visualization)
```
**ACTION:** Generate those plots with the following command and then open the visualization.

In [ ]:
qiime taxa barplot \
  --i-table table-dada2.qza \
  --i-taxonomy taxonomy-dada2.qza \
  --m-metadata-file mappingfile_soil2018.tsv \
  --o-visualization taxa-bar-plots-dada2.qzv


### Output artifacts

taxa-bar-plots.qzv


### D. Viewing our `.qzv` file

We are not working in an environment that allows us to see interactively our visualization files in this notebook. We need to download the `.qzv` files to our local computer to view them.

1. Go to the Home tab.
2. To download the `.qzv` files to your computer, find the `.qzv` file and right-click. The `Download` option will become available.
3. Launch the website: https://view.qiime2.org/.
4. Drag and Drop `.qzv` file to view plot.

> Make a new directory / folder on your local computer desktop to store all the `.qza` and `.qzv` files that are generated through this tutorial.

Next, let's **PRACTICE:**
> 1. Visualize the samples at Level 2 (which corresponds to the phylum level in this analysis). First, sort the samples by Treatment. Then sort  by Location. Lastly, sort by pH. 
>
> 2. What are the dominant phyla in each in Treatment? in Location? in pH?
>
> 3. Do you observe any consistent change across the samples?

### E. Run against the SILVA database

The GreenGenes Database is not currently annotated as much as the SILVA Database. This takes a longer amount of time and goes through similar steps as preparing the `taxonomy.qza` file.

_**Running the SIlVA Database comparison requires a long time, 10+ hours.**_

We will need to submit the job in a different format than in a code cell. If we close the notebook, all jobs end. We will move into the `Terminal` within our Jupyter Notebook to submit the job and it will keep running in the background.

1. Go to the home menu.
2. Go to the right side of the screen towards the `New` button and select `Terminal`.
3. Type `screen`.
4. Press the `enter` key twice. You will see the black screen and a command prompt `$`.
5. After the command prompt `$` type `pwd`.
6. We need to be in the directory `/scratch/2018`. If you are in root `/`. you can type `cd /scratch/2018`.
7. Type `pwd` to see if you are in the `/scratch/2018` directory.
8. If you are in the `/scratch/2018` directory, type the command below or copy to the command prompt `$`:

**ACTION:** Using the `wget` command we can download the `feature-classifer` from QIIME 2:

In [ ]:
wget -O "silva-119-99-515-806-nb-classifier.qza" "https://data.qiime2.org/2018.11/common/silva-132-99-515-806-nb-classifier.qza"

**ACTION:** We will run the same set-up as we did earlier but will substitute the `silva` database:

1. Run in terminal for this!
2. Navigate to scratch directory
3. Run alias
```
alias qiime='docker run -t -i -u $(id -u):$(id -g) -v $(pwd):/data  -v /scratch/tmp:/tmp qiime2/core:2018.11 qiime'
```
4. Submit command below:

In [ ]:
qiime feature-classifier classify-sklearn \
--i-classifier silva-119-99-515-806-nb-classifier.qza \
--i-reads rep-seqs-dada2.qza \
--o-classification taxonomy-dada2-silva.qza

**ACTION:** The `silva database` requires different steps to provide taxonomy to the sequences. Follow the next steps to generate these:

In [ ]:
qiime tools export taxonomy-dada2-silva.qza \
--output-dir taxonomy-with-spaces

**ACTION:** Here we will take the taxonomy and generate a `.qzv` visualization file:

In [ ]:
qiime metadata tabulate \
--m-input-file taxonomy-with-spaces/taxonomy.tsv  \
--o-visualization taxonomy-as-metadata.qzv

**ACTION:** The file needs to be exported in a format using the command below:

In [ ]:
qiime tools export taxonomy-as-metadata.qzv \
--output-dir taxonomy-as-metadata

**ACTION:** The file will be imported back into a QIIME 2 artifact that we can work with downstream:

In [ ]:
qiime tools import \
--type 'FeatureData[Taxonomy]' \
--input-path taxonomy-as-metadata/metadata.tsv \
--output-path taxonomy-without-spaces-silva.qza

#### Output artifacts
```
taxonomy-dada2-silva.qza
taxonomy-without-spaces-silva.qza
```
#### Output visualization
```
taxonomy-as-metadata.qzv
```

Next, we can view the taxonomic composition of our samples with interactive bar plots.

**ACTION:** Generate those plots with the following command and then open the visualization.

In [ ]:
qiime taxa barplot \
--i-table table-dada2.qza \
--i-taxonomy taxonomy-silva.qza \
--m-metadata-file metadata-flc-2019.tsv \
--o-visualization taxa-bar-plots-silva.qzv

#### Output visualization
```
taxa-bar-plots-silva.qzv
```
### F. Viewing our `.qzv` file

> Make a new directory / folder on your local computer desktop to store all the `.qza` and `.qzv` files that are generated through this tutorial.

Now let's **PRACTICE:**
> Compare the taxonomic assignment differences between the SILVA and GreenGenes output. Is there a difference in the percent of "unknown" taxa?

### G. Wrap up
Don't forget to transfer data to the CyVerse Data Store and to suspend your instance to avoid using Atmospheric Units (AUs) and the instance being 'Shelved'.


## Module: How do my samples compare to one another? How similar or different are they to each other?

In this section, we want to understand the diversity and richness within each of our samples and between each of the samples. 

### A. Diversity analysis
QIIME 2’s diversity analyses are available through the `diversity` plugin, which supports computing alpha and beta diversity metrics, applying related statistical tests, and generating interactive visualizations. We’ll first apply the `core-metrics-phylogenetic`method, which rarefies a `FeatureTable[Frequency]` to a user-specified depth, computes several alpha and beta diversity metrics, and generates principle coordinates analysis (PCoA) plots using Emperor for each of the beta diversity metrics. The metrics computed by default are:

*Alpha diversity*

Each of these takes into consideration measurements with quantitative and qualitative inputs. Further explanations can be found on QIIME 2 for more information:

- Shannon’s diversity index (a quantitative measure of community richness)

- Observed OTUs (a qualitative measure of community richness)

- Faith’s Phylogenetic Diversity (a qualitiative measure of community richness that incorporates phylogenetic relationships between the features)

- Evenness (or Pielou’s Evenness; a measure of community evenness)

*Beta diversity*

- Jaccard distance (a qualitative measure of community dissimilarity)

- Bray-Curtis distance (a quantitative measure of community dissimilarity)

- unweighted UniFrac distance (a qualitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)

- weighted UniFrac distance (a quantitative measure of community dissimilarity that incorporates phylogenetic relationships between the features)

```
Plugin: diversity core-metrics-phylogenetic
Input:  rooted-tree-dada2.qza; table.qza; mappingfile.tsv
Output: many artifact files
```

An important parameter that needs to be provided to this script is `--p-sampling-depth`, which is the even sampling (i.e. rarefaction) depth. Because most diversity metrics are sensitive to different sampling depths across different samples, this script will randomly subsample the counts from each sample to the value provided for this parameter. 

> For example, if you provide `--p-sampling-depth` 500, this step will subsample the counts in each sample without replacement so that each sample in the resulting table has a total count of 500. If the total count for any sample(s) are smaller than this value, those samples will be dropped from the diversity analysis. Choosing this value is tricky. We recommend making your choice by reviewing the information presented in the table.qzv file that was created above and choosing a value that is as high as possible (so you retain more sequences per sample) while excluding as few samples as possible.


In the total samples for this data set, the lowest amount of sequences found in the sample is **16377**. We will use this number for our `--p-sampling-depth`.

#### This value was chosen based on the number of sequences in the non-control samples because it’s close to the number of sequences in the next few samples that have higher sequence counts, and because it is considerably higher (relatively) than the number of sequences in the one sample that has fewer sequences.

This will allow us to retain most of our samples, a few of our negative controls are removed. This helps increase more sequences from other samples. Alternatively, the sample depth of 2279 will sample less sequences but include all samples. The one sample that has fewer sequences will be dropped from the core-metrics-phylogenetic analyses and anything that uses these results. Conduct both sampling depths and compare how this affects the results.

_Note
In many Illumina runs you’ll observe a few samples that have very low sequence counts. You will typically want to exclude those from the analysis by choosing a larger value for the sampling depth at this stage._

In [ ]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree-dada2.qza \
  --i-table table-dada2.qza \
  --p-sampling-depth 21000 \
  --m-metadata-file metadata-flc-2019.tsv \
  --output-dir core-metrics-results-21000

#### Output artifacts
```
core-metrics-results/rarefied_table.qza
core-metrics-results/faith_pd_vector.qza
core-metrics-results/observed_otus_vector.qza
core-metrics-results/shannon_vector.qza
core-metrics-results/evenness_vector.qza
core-metrics-results/unweighted_unifrac_distance_matrix.qza
core-metrics-results/weighted_unifrac_distance_matrix.qza
core-metrics-results/jaccard_distance_matrix.qza
core-metrics-results/bray_curtis_distance_matrix.qza
core-metrics-results/bray_curtis_pcoa_results.qza
core-metrics-results/unweighted_unifrac_pcoa_results.qza
core-metrics-results/weighted_unifrac_pcoa_results.qza
core-metrics-results/jaccard_pcoa_results.qza
```
#### Output visualizations
```
core-metrics-results/unweighted_unifrac_emperor.qzv
core-metrics-results/weighted_unifrac_emperor.qzv
core-metrics-results/jaccard_emperor.qzv
core-metrics-results/bray_curtis_emperor.qzv
```

After computing diversity metrics, we can begin to explore the microbial composition of the samples in the context of the sample metadata. This information is present in the sample metadata file that was downloaded earlier.

We’ll test for associations between categorical metadata columns and alpha diversity data.

```
qiime diversity alpha-group-significance \
--i-alpha-diversity vector.qza \
--m-metadata-file metadata.tsv \
--o-visualization group-significance.qzv
```

First for the Faith Phylogenetic Diversity (a measure of community richness) metric:

In [ ]:
qiime diversity alpha-group-significance \
--i-alpha-diversity core-metrics-results-21000/faith_pd_vector.qza \
--m-metadata-file metadata-flc-2019.tsv \
--o-visualization core-metrics-results-21000/faith-pd-group-significance.qzv

Second for the evenness metrics:

Evenness (or Pielou’s Evenness; a measure of community evenness)

In [ ]:
qiime diversity alpha-group-significance \
--i-alpha-diversity core-metrics-results-21000/evenness_vector.qza \
--m-metadata-file metadata-flc-2019.tsv \
--o-visualization core-metrics-results-21000/evenness-group-significance.qzv

Third for Shannon Diversity:

Shannon’s diversity index (a quantitative measure of community richness)

In [ ]:
qiime diversity alpha-group-significance \
--i-alpha-diversity core-metrics-results-16377/shannon_vector.qza \
--m-metadata-file mappingfile_soil2018.tsv \
--o-visualization core-metrics-results-16377/shannon-group-significance.qzv

Fourth for observed OTUs:

Observed OTUs (a qualitative measure of community richness)

In [ ]:
qiime diversity alpha-group-significance \
--i-alpha-diversity core-metrics-results-21000/observed_otus_vector.qza \
--m-metadata-file mappingfile_soil2018.tsv \
--o-visualization core-metrics-results-21000/observed_otus-group-significance.qzv

#### Output visualization
```
core-metrics-results/evenness-group-significance.qzv
core-metrics-results/faith-pd-group-significance.qzv
core-metrics-results/observed_otus-group-significance.qzv
core-metrics-results/shannon-group-significance.qzv
```

> Question
>
> What discrete sample metadata categories are most strongly associated with the differences in microbial community **richness**? Are these differences statistically significant?
>
> Question
>
> What discrete sample metadata categories are most strongly associated with the differences in microbial community **evenness**? Are these differences statistically significant?
>
> **PRACTICE:**
>
>1. What sample metadata categories (i.e.location) are most strongly associated with the differences in microbial community richness? Remember when looking at these plots, the 
>
>2. Are these differences statistically significant? (look at the p-value but understand the number of replicates that can impact the p-value)
>

### B. Statistical Analysis 

Next we’ll analyze sample composition in the context of discrete metadata using PERMANOVA [first described in Anderson (2001)] using the `beta-group-significance` command. 

The following commands will test whether distances between samples within a group, such as samples from the same treatment (e.g., mulch), are more similar to each other then they are to samples from the other groups (e.g., soil, storebrand). 

If you call this command with the `--p-pairwise parameter`, as we’ll do here, it will also perform pairwise tests that will allow you to determine which specific pairs of groups (e.g., soil and store brand) differ from one another, if any. This command can be slow to run, especially when passing `--p-pairwise`, since it is based on permutation tests. So, unlike the previous commands, we’ll run this on specific categories of metadata that we’re interested in exploring, rather than all metadata categories that it’s applicable to. 

Here we’ll apply this to our unweighted UniFrac distances, using two sample metadata categories, as follows.

Using the headings for the metadata table: `Treatment`, `pH`, and `temperatureC`.

**ACTION:** Run the `diversity beta-group-significance` for the metadata column Treatment:

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix core-metrics-results-21000/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file metadata-flc-2019.tsv \
--m-metadata-column Treatment \
--o-visualization core-metrics-results-21000/Treatment-unweighted-unifrac-samples.qzv \
--p-pairwise

**ACTION:** Run the `diversity beta-group-significance` for the metadata column pH:

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix core-metrics-results-21000/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file metadata-flc-2019.tsv \
--m-metadata-column pH \
--o-visualization core-metrics-results-21000/pH-unweighted_unifrac_distance_matrix.qzv \
--p-pairwise

**ACTION:** Run the `diversity beta-group-significance` for temperatureC.

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix core-metrics-results-21000/unweighted_unifrac_distance_matrix.qza \
--m-metadata-file metadata-flc-2019.tsv \
--m-metadata-column temperatureC \
--o-visualization core-metrics-results-21000/temperatureC-unweighted_unifrac_distance_matrix.qzv \
--p-pairwise

#### Output visualization
```
16377-core-metrics-results/Treatment-unweighted-unifrac-samples.qzv
16377-core-metrics-results/location-unweighted_unifrac_distance_matrix.qzv
16377-core-metrics-results/water-source-unweighted_unifrac_distance_matrix.qzv
```

### C. Viewing our `.qzv` file

We are not working in an environment that allows us to see interactively our visualization files in this notebook. We need to download the `.qzv` files to our local computer to view them.

1. Go to the Home tab.
2. To download the `.qzv` files to your computer, find the `.qzv` file and right-click. The `Download` option will become available.
3. Launch the website: https://view.qiime2.org/.
4. Drag and Drop `.qzv` file to view plot.

> Make a new directory / folder on your local computer desktop to store all the `.qza` and `.qzv` files that are generated through this tutorial.
>
> **PRACTICE:**
>
>A. Are the associations between subjects and differences in microbial composition statistically significant? 
>
>
>B.  How about `Treatment` sites? What specific pairs of `Treatment` sites are significantly different from each other?
>
>
>C. Ask the same about the `pH` and `TemperatureC`.

Again, none of the continuous sample metadata that we have for this data set are correlated with sample composition, so we won’t test for those associations here. If you’re interested in performing those tests, you can use the `qiime metadata distance-matrix` in combination with `qiime diversity mantel` and `qiime diversity bioenv` commands.

### D. Beta Diversity Plotting

Finally, ordination is a popular approach for exploring microbial community composition in the context of sample metadata. We can use the Emperor tool to explore principal coordinates (PCoA) plots in the context of sample metadata. While our core-metrics-phylogenetic command did already generate some Emperor plots, we want to pass an optional parameter, `--p-custom-axis`, which is very useful for exploring time series data. 

The PCoA results that were used in `core-metrics-phylogeny` are also available, making it easy to generate new visualizations with Emperor. We will generate Emperor plots for `unweighted UniFrac` and `Bray-Curtis` so that the resulting plot will contain axes for principal coordinate 1, principal coordinate 2 and principle coordinate 3. We will use that last axis to explore how these samples changed over time.

**ACTION:** Generate the emperor plots for the unweighted-unifrac

In [ ]:
qiime emperor plot \
--i-pcoa core-metrics-results-21000`/unweighted_unifrac_pcoa_results.qza \
--m-metadata-file metadata-flc-2019.tsv \
--o-visualization core-metrics-results-21000/unweighted-unifrac-samples.qzv

**ACTION:** Generate the emperor plots for bray-curtis

In [ ]:
qiime emperor plot \
--i-pcoa core-metrics-results-21000/bray_curtis_pcoa_results.qza \
--m-metadata-file metadata-flc-2019.tsv \
--o-visualization core-metrics-results-21000/bray-curtis-emperor.qzv

#### Output visualization
```
core-metrics-results/bray-curtis-emperor.qzv
core-metrics-results/unweighted-unifrac-emperor.qzv
```

### E. Viewing our `unweighted-unifrac-emperor.qzv` file

We are not working in an environment that allows us to see interactively our visualization files. 

1. Go to the Home tab.
2. Download the `.qzv` files to your computer.
3. Launch https://view.qiime2.org/.
4. Drag and Drop `.qzv` file to view plot.

> **PRACTICE:**
>
> A. Do the Emperor plots support the other beta diversity analyses we’ve performed here? (Hint: Experiment with coloring points by different metadata.)
>
> B. What differences do you observe between the unweighted UniFrac and Bray-Curtis PCoA plots?
>

### F. Alpha rarefaction plotting

In this section we’ll explore alpha diversity as a function of sampling depth using the `qiime diversity alpha-rarefaction` visualizer. This visualizer computes one or more alpha diversity metrics at multiple sampling depths, in steps between 1 (optionally controlled with `--p-min-depth`) and the value provided as `--p-max-depth`. At each sampling depth step, 10 rarefied tables will be generated, and the diversity metrics will be computed for all samples in the tables. The number of iterations (rarefied tables computed at each sampling depth) can be controlled with `--p-iterations`. Average diversity values will be plotted for each sample at each even sampling depth, and samples can be grouped based on metadata in the resulting visualization if sample metadata is provided with the `--m-metadata-file` parameter.

**ACTION:** Run the `diversity alpha-rarefaction` plugin:

In [ ]:
qiime diversity alpha-rarefaction \
  --i-table table-dada2.qza \
  --i-phylogeny rooted-tree-dada2.qza \
  --p-max-depth 4000 \
  --m-metadata-file metadata-flc-2019.tsv \
  --o-visualization alpha-rarefaction-dada2.qzv

#### Output visualization
```
alpha-rarefaction.qzv
```

The visualization will have two plots:

The _top plot_ is an alpha rarefaction plot, and is primarily used to determine if the richness of the samples has been fully observed or sequenced. If the lines in the plot appear to “level out” (i.e., approach a slope of zero) at some sampling depth along the x-axis, that suggests that collecting additional sequences beyond that sampling depth would not be likely to result in the observation of additional features. If the lines in a plot don’t level out, this may be because the richness of the samples hasn’t been fully observed yet (because too few sequences were collected), or it could be an indicator that a lot of sequencing error remains in the data (which is being mistaken for novel diversity).

The _bottom plot_ in this visualization is important when grouping samples by metadata. It illustrates the number of samples that remain in each group when the feature table is rarefied to each sampling depth. If a given sampling depth d is larger than the total frequency of a sample s (i.e., the number of sequences that were obtained for sample s), it is not possible to compute the diversity metric for sample s at sampling depth d. If many of the samples in a group have lower total frequencies than d, the average diversity presented for that group at d in the top plot will be unreliable because it will have been computed on relatively few samples. When grouping samples by metadata, it is therefore essential to look at the bottom plot to ensure that the data presented in the top plot is reliable.

> **NOTE:**
>
> The value that you provide for `--p-max-depth` should be determined by reviewing the “Frequency per sample” information presented in the `table.qzv` file that was created above. In general, choosing a value that is somewhere around the median frequency seems to work well, but you may want to increase that value if the lines in the resulting rarefaction plot don’t appear to be leveling out, or decrease that value if you seem to be losing many of your samples due to low total frequencies closer to the minimum sampling depth than the maximum sampling depth.
>
> **QUESTION:**
>
> When grouping samples by “Treatment” and viewing the alpha rarefaction plot for the “observed_otus” metric, which garden, non-garden, mulch (if any) appear to exhibit sufficient diversity coverage (i.e., their rarefaction curves level off)?


## You have finished the analysis of this Jupyter Notebook!

### G. Wrap up
Don't forget to transfer data to the CyVerse Data Store and to suspend your instance to avoid using Atmospheric Units (AUs) and the instance being 'Shelved'.

### Module: Option 2 for denoising and clustering reads using Deblur method

[Deblur](http://msystems.asm.org/content/2/2/e00191-16) uses sequence error profiles to associate erroneous sequence reads with the true biological sequence from which they are derived, resulting in high quality sequence variant data. This is applied in two steps. First, an initial quality filtering process based on quality scores is applied. This method is an implementation of the quality filtering approach described by [Bokulich et al. (2013)](http://www.nature.com/nmeth/journal/v10/n1/abs/nmeth.2276.html).

In [ ]:
qiime quality-filter q-score \
 --i-demux demux.qza \
 --o-filtered-sequences demux-filtered.qza \
 --o-filter-stats demux-filter-stats.qza

#### Output visualizations:

```
demux-filter-stats.qzv
deblur-stats.qzv
```

In [ ]:
qiime deblur denoise-16S \
  --i-demultiplexed-seqs demux-filtered.qza \
  --p-trim-length 245 \
  --o-representative-sequences rep-seqs-deblur.qza \
  --o-table table-deblur.qza \
  --p-sample-stats \
  --o-stats deblur-stats.qza

Output artifacts:

```
deblur-stats.qza
table-deblur.qza
rep-seqs-deblur.qza
```

> **Note:**
>
> The two commands used in this section generate QIIME 2 artifacts containing summary statistics. To view those summary statistics, you can visualize them using qiime metadata tabulate and qiime deblur visualize-stats, respectively:

In [ ]:
qiime metadata tabulate \
  --m-input-file demux-filter-stats.qza \
  --o-visualization demux-filter-stats.qzv
qiime deblur visualize-stats \
  --i-deblur-stats deblur-stats.qza \
  --o-visualization deblur-stats.qzv

Output visualizations:
```
demux-filter-stats.qzv
deblur-stats.qzv
```

> **PRACTICE**
>
> Compare the difference between DADA2 and Deblur.